In [76]:
import advent

data = advent.get_char_grid(23, 'txt')

In [77]:
def walkable(node: tuple[int, int]) -> bool:
    return node[0] >= 0 and node[0] < data.shape[0] and node[1] >= 0 and node[1] < data.shape[1] and data[node] != '#'

def adjacent(node: tuple[int, int, int, int]) -> list[tuple[tuple[int, int, int, int], int]]:
    # To prevent stepping back and forth, a node is 4 numbers: current and last
    px, py, x, y = node
    result: list[tuple[tuple[int, int, int, int], int]] = []
    if walkable((x-1, y)) and data[x-1, y] != 'v' and (x-1, y) != (px, py): result.append(((x, y, x-1, y), -1))
    if walkable((x+1, y)) and data[x+1, y] != '^' and (x+1, y) != (px, py): result.append(((x, y, x+1, y), -1))
    if walkable((x, y-1)) and data[x, y-1] != '>' and (x, y-1) != (px, py): result.append(((x, y, x, y-1), -1))
    if walkable((x, y+1)) and data[x, y+1] != '<' and (x, y+1) != (px, py): result.append(((x, y, x, y+1), -1))
    return result

In [78]:
# Unfortunately, cannot do this with dijkstra/heapq
# It can be done similarly to dijkstra, by just brute forcing it
# that is, remove all the optimizations and reasons why dijkstra is fast
from typing import Any, Callable
from tqdm.notebook import tqdm

Node = Any

def solve_maze(start: Node, adjacent: Callable[[Node], list[tuple[Node, int]]]):
    f: list[tuple[int, Node]]= [(0, start)] # [(g, n)]
    open: dict[Node, int] = {} # track longest routes so far
    open_parents: dict[Node, Node] = {start: start}
    # We have no closed because we are not doing dijkstra

    progress = 0
    with tqdm(total=data.shape[0] ** 2) as pbar:
        while f:
            current_g, current_n = f.pop(0)
            for adj, adj_g in adjacent(current_n):
                new_g = current_g + adj_g
                if adj not in open or open[adj] > new_g:
                    # We found a new longest route
                    f.append((new_g, adj))
                    open[adj] = new_g
                    open_parents[adj] = current_n

            progress_tmp = len(open)
            #print(progress_tmp)
            if progress_tmp > progress: 
                pbar.update(progress_tmp - progress)  # noqa
                progress = progress_tmp
        
    return open, open_parents



In [79]:

start = (0, 0, 0, 1)
open, open_parents = solve_maze(start, adjacent)
for x in open:
    if x[2] == data.shape[0]-1: # The ending node
        print(open[x] * -1)

  0%|          | 0/19881 [00:00<?, ?it/s]

2386


In [80]:
# Path is the previous, as well as a list of intersections
Point = tuple[int, int]
Path = tuple[Point, Point, tuple[Point, ...]]

# The problem with doing a 'naive approach' where you just store the entire
# path and check if the node is in the path: its too slow :(
# Instead, let's only store the intersections

def is_intersection(x: int, y:int):
    if x == 0 or x == data.shape[0] - 1: return True
    return ((data[x-1, y] == '#') + \
            (data[x+1, y] == '#') + \
            (data[x, y-1] == '#') + \
            (data[x, y+1] == '#')) < 2

# Add x, y to node to create a new Node. Assume x, y is walkable
# If x, y is an already visited intersection, return None
def create_new_node(node: Path, x: int, y: int, prev: Point) -> Path|None:
    if is_intersection(x, y):
        if (x, y) in node[2]: return None
        return (prev, (x, y), (*node[2], (x, y)))
    else: return (prev, (x, y), node[2])

def adjacent_path(node: Path) -> list[tuple[Path, int]]:
    x, y = node[1]
    result: list[tuple[Path, int]] = []
    if walkable((x-1, y)) and (x-1, y) != node[0]:
        newnode = create_new_node(node, x-1, y, (x, y))
        if newnode: result.append((newnode, -1))
    if walkable((x+1, y)) and (x+1, y) != node[0]:
        newnode = create_new_node(node, x+1, y, (x, y))
        if newnode: result.append((newnode, -1))
    if walkable((x, y-1)) and (x, y-1) != node[0]:
        newnode = create_new_node(node, x, y-1, (x, y))
        if newnode: result.append((newnode, -1))
    if walkable((x, y+1)) and (x, y+1) != node[0]:
        newnode = create_new_node(node, x, y+1, (x, y))
        if newnode: result.append((newnode, -1))
    return result

In [81]:
start = ((0, 1), (0, 1), ((0, 1),))
open, _ = solve_maze(start, adjacent_path)
longest = 0
for x in open:
    if x[1][0] == data.shape[0]-1: # The ending node
        if open[x] < longest:
            longest = open[x]
print(-1 * longest)

  0%|          | 0/19881 [00:00<?, ?it/s]

KeyboardInterrupt: 